In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import PIL.Image as img
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/Covid19-dataset'
train_dir='/content/drive/MyDrive/Covid19-dataset/train'
test_dir='/content/drive/MyDrive/Covid19-dataset/test'

Mounted at /content/drive


# **32 BS 50 Epoch 0.88**

In [7]:
model = Sequential()
model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [8]:
train_data = ImageDataGenerator(rescale = 1/255.0 , validation_split=0.2)
test_data = ImageDataGenerator(rescale = 1/255.0)

train_set = train_data.flow_from_directory(
                    train_dir,
                    subset = 'training',
                    batch_size=32,
                    target_size=(224,224),
                    class_mode = 'sparse'
)
val_set = train_data.flow_from_directory(
                    train_dir,
                    subset = 'validation',
                    batch_size=32,
                    target_size=(224,224),
                    class_mode = 'sparse'
)
test_set = test_data.flow_from_directory(
                    test_dir,
                    batch_size=32,
                    target_size=(224,224),
                    class_mode = 'sparse'
)

Found 201 images belonging to 3 classes.
Found 50 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_set, epochs=50, validation_data=val_set)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_set)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

Epoch 1/50
7/7 [==============================] - 57s 8s/step - loss: 1.4076 - accuracy: 0.3383 - val_loss: 1.1087 - val_accuracy: 0.4400
Epoch 2/50
7/7 [==============================] - 51s 7s/step - loss: 1.0884 - accuracy: 0.4428 - val_loss: 1.0633 - val_accuracy: 0.4400
Epoch 3/50
7/7 [==============================] - 48s 7s/step - loss: 1.0496 - accuracy: 0.4428 - val_loss: 0.9616 - val_accuracy: 0.4400
Epoch 4/50
7/7 [==============================] - 45s 6s/step - loss: 0.8883 - accuracy: 0.6119 - val_loss: 1.0447 - val_accuracy: 0.4600
Epoch 5/50
7/7 [==============================] - 46s 6s/step - loss: 0.5737 - accuracy: 0.7413 - val_loss: 0.9273 - val_accuracy: 0.6600
Epoch 6/50
7/7 [==============================] - 41s 6s/step - loss: 0.5320 - accuracy: 0.7761 - val_loss: 0.6854 - val_accuracy: 0.7200
Epoch 7/50
7/7 [==============================] - 45s 6s/step - loss: 0.4628 - accuracy: 0.8408 - val_loss: 0.6285 - val_accuracy: 0.7800
Epoch 8/50
7/7 [==================

In [11]:
from sklearn.metrics import precision_score, recall_score, f1_score
# Get the predicted labels for the validation set
y_pred = model.predict(val_set)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true labels for the validation set
y_true = val_set.labels

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Print the metrics
print("Accuracy:", test_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

2/2 [==============================] - 7s 1s/step
Accuracy: 0.8787878751754761
Precision: 0.40973262032085567
Recall: 0.42
F1-Score: 0.4131870967741935
